In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
#from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [ ]:
SETTINGS_NO = "Case8"

In [ ]:
from google.colab import files

# This will prompt you to upload a file from your local system
uploaded = files.upload()
filename = next(iter(uploaded))  # Get the first (and only) filename

Saving FinalDataset_transposed_02282025.csv to FinalDataset_transposed_02282025.csv


In [ ]:
#DATASET_NAME = "FinalDataset_transposed_02282025.csv"
DATASET_NAME =filename
df = pd.read_csv(DATASET_NAME, encoding='ISO-8859-1')

In [ ]:
def clean_text(text):
    # Remove numbers and special characters
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [ ]:
unique_scc1_values = df['SCC2'].unique()
pivot_df = df
pivot_df['SCC1'] = pd.factorize(pivot_df['SCC1'])[0]
pivot_df['SCC3'] = pd.factorize(pivot_df['SCC3'])[0]

In [ ]:
# Step 2: Ensure SCC2 is treated as a string column
pivot_df['SCC2'] = pivot_df['SCC2'].astype(str)

# Split SCC2 based on ;
split_SCC2 = pivot_df['SCC2'].str.split(';', expand=True)

# Get unique sentence fragments
unique_fragments = pd.unique(split_SCC2.values.ravel('K'))

print(unique_fragments)

# Remove None values (if any)
unique_fragments = [fragment for fragment in unique_fragments if pd.notna(fragment) and fragment != 'nan']

# Create one-hot encoding for each fragment
one_hot_encoded = pd.DataFrame(0, index=pivot_df.index, columns=unique_fragments)

print(one_hot_encoded)

for col in split_SCC2.columns:
    for idx, fragment in split_SCC2[col].items():
        if pd.notna(fragment) and fragment != 'nan':
            one_hot_encoded.at[idx, fragment] = 1

# Step 3: Convert each row of one-hot encoding into a tuple
one_hot_tuples = [tuple(row) for row in one_hot_encoded.values]

# Step 4: Assign a unique sequential number to each unique combination
unique_combinations = pd.factorize(one_hot_tuples)[0]

# Add the sequential numbers to the DataFrame
pivot_df['SCC2'] = unique_combinations

# Display the transformed DataFrame
print(pivot_df)

['Miscellaneous' 'Electric Generation' 'Boiler' 'Agriculture' 'Road'
 'Mobile' 'Petrochemical' 'Biomass Burning' 'Waste' 'Geological' 'Desert'
 'Volcanic' 'Construction' 'Non-industrial' 'Industrial' 'Windblown'
 'Geothermal' 'Regional' 'Residential Heating' 'Cooking'
 'Waste Combustion' 'Mineral Products' 'Turbine' 'Process Heater'
 'Cigarette' 'Surface Coating' 'Pulp And Paper' 'Residential'
 'Stationary Engine' 'Furnace' 'Domestic' 'Chemical Manufacturing'
 'Combustion' 'Refinery' 'Incinerator' 'Secondary Aluminum Furnace'
 'Melting Pot' 'Smelting' 'Crop Heating' 'Food And Agriculture' None
 ' Boiler' ' Onroad' ' Catalytic Cracker' ' Residential Wood Combustion'
 ' Agriculture' ' Incinerator' ' Wildfire' ' Charbroiling' ' Kiln'
 ' Stirfrying' ' Process Heater' ' Frying' ' Spraying' ' Marine'
 ' Furnace' ' Aircraft' ' Metal' ' Silo' ' Gas Turbine' ' Prescribed Fire'
 ' Metal And Construction' ' Wood Products' ' Shredding' ' Mining'
 ' Calciner' ' Space Heater And Water Heater' ' Spac

<ipython-input-7-186c2e49aae4>:29: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_combinations = pd.factorize(one_hot_tuples)[0]


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer

print(pivot_df.columns[3:])

# imputer = IterativeImputer()
#imputer = SimpleImputer(strategy='median')  #  option 1
imputer = KNNImputer() # option 2
df_imputed = pd.DataFrame(imputer.fit_transform(pivot_df.iloc[:,3:]), columns=df.columns[3:])
pivot_df[pivot_df.columns[3:]]=df_imputed

# # Opton 3 for minimum
# min_values = pivot_df.iloc[:, 3:].min()
# imputers = {col: SimpleImputer(strategy='constant', fill_value=min_val) for col, min_val in min_values.items()}
# # Apply imputation column by column
# for col in pivot_df.columns[3:]:
#     pivot_df[[col]] = imputers[col].fit_transform(pivot_df[[col]])

pivot_df



Index(['SCC1', 'SCC2', 'SCC3', 'Aluminum', 'Ammonium', 'Antimony', 'Arsenic',
       'Barium', 'Cadmium', 'Calcium', 'Chloride ion', 'Chromium', 'Cobalt',
       'Copper', 'Elemental Carbon', 'Indium', 'Iron', 'Lead', 'Magnesium',
       'Manganese', 'Mercury', 'Molybdenum', 'Nickel', 'Nitrate',
       'Organic carbon', 'Phosphorus', 'Potassium', 'Potassium ion',
       'Selenium', 'Silicon', 'Silver', 'Sodium', 'Sodium ion', 'Strontium',
       'Sulfate', 'Sulfur', 'Tin', 'Titanium', 'Vanadium', 'Zinc',
       'Zirconium'],
      dtype='object')


,Unnamed: 0,PROFILE_CODE,PROFILE_NAME,SCC1,SCC2,SCC3,Aluminum,Ammonium,Antimony,Arsenic,...,Sodium,Sodium ion,Strontium,Sulfate,Sulfur,Tin,Titanium,Vanadium,Zinc,Zirconium
0,0,2.5,Overall Composite,0.0,0.0,0.0,4.4430,0.0220,0.765000,1.4760,...,1.652,0.25,0.01400,5.5140,2.9720,0.1170,0.1940,0.0610,2.2330,0.002000
1,1,3190.0,Coal Combustion,1.0,1.0,1.0,5.9680,0.3476,0.000000,0.0000,...,0.220,0.25,0.19640,10.1716,2.9480,0.0000,0.4315,0.0000,0.0797,0.024700
2,2,3191.0,Coal Combustion,1.0,1.0,1.0,6.3904,0.3354,0.000000,0.0000,...,0.220,0.25,0.13730,2.1195,0.5934,0.0000,0.4018,0.0000,0.0857,0.022900
3,3,3192.0,Coal Combustion,1.0,1.0,1.0,5.5457,0.3598,0.000000,0.0000,...,0.220,0.25,0.25560,18.2238,5.3025,0.0000,0.4612,0.0000,0.0737,0.026600
4,4,3193.0,Coal Combustion,1.0,1.0,1.0,6.9752,0.3412,0.000000,0.0000,...,0.220,0.25,0.27840,7.5760,2.2323,0.0000,0.4532,0.0000,0.0653,0.026100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,1341,900122.5,Clay And Fly Ash Sintering - Average,5.0,81.0,0.0,0.4650,0.1600,0.003435,0.0008,...,0.520,0.25,0.00575,9.6000,3.2000,0.0033,0.0070,0.0040,0.0090,0.002852
1342,1342,900132.5,Mineral Products - Average,2.0,50.0,0.0,2.1110,0.1600,0.021000,0.0080,...,1.466,0.25,0.01200,14.0610,5.1430,0.0040,0.1340,0.0040,0.1270,0.001000
1343,1343,900142.5,Petroleum Industry - Average,7.0,27.0,0.0,2.4840,0.1600,0.001000,0.0008,...,0.220,0.25,0.02800,23.5000,10.0560,0.0010,0.0590,0.0061,0.2780,0.025000
1344,1344,900152.5,Pulp And Paper Industry,1.0,47.0,0.0,0.0900,0.1600,0.004000,0.0010,...,10.775,0.25,0.00700,20.3960,5.6700,0.0080,0.0030,0.0090,0.0170,0.003000


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:
def assign_cluster_name(cluster):
    names = pivot_df[pivot_df['speciated_cluster'] == cluster]['PROFILE_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

In [ ]:
com_map = {}
loop_info = []
#parameters
StartCol=3
k_SVD = 16
K_knn = 5

In [ ]:
#X = pivot_df.iloc[:, 3:]
## scaler = StandardScaler()
#scaler = MinMaxScaler()
#X = scaler.fit_transform(X)

In [ ]:
#pip install cdlib

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from scipy.sparse.linalg import svds

import cdlib
from cdlib import algorithms
import networkx as nx

In [ ]:
X = pivot_df.iloc[:, 6:]
X

,Aluminum,Ammonium,Antimony,Arsenic,Barium,Cadmium,Calcium,Chloride ion,Chromium,Cobalt,...,Sodium,Sodium ion,Strontium,Sulfate,Sulfur,Tin,Titanium,Vanadium,Zinc,Zirconium
0,4.4430,0.0220,0.765000,1.4760,0.05200,0.4210,2.6140,0.17,0.1030,0.031000,...,1.652,0.25,0.01400,5.5140,2.9720,0.1170,0.1940,0.0610,2.2330,0.002000
1,5.9680,0.3476,0.000000,0.0000,1.33150,0.0000,3.4536,0.17,0.0176,0.000000,...,0.220,0.25,0.19640,10.1716,2.9480,0.0000,0.4315,0.0000,0.0797,0.024700
2,6.3904,0.3354,0.000000,0.0000,0.36970,0.0000,2.5607,0.17,0.0149,0.000000,...,0.220,0.25,0.13730,2.1195,0.5934,0.0000,0.4018,0.0000,0.0857,0.022900
3,5.5457,0.3598,0.000000,0.0000,2.29330,0.0000,4.3464,0.17,0.0202,0.000000,...,0.220,0.25,0.25560,18.2238,5.3025,0.0000,0.4612,0.0000,0.0737,0.026600
4,6.9752,0.3412,0.000000,0.0000,1.23480,0.0000,4.6981,0.17,0.0198,0.000000,...,0.220,0.25,0.27840,7.5760,2.2323,0.0000,0.4532,0.0000,0.0653,0.026100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,0.4650,0.1600,0.003435,0.0008,0.05025,0.0022,15.0700,0.17,0.0040,0.000995,...,0.520,0.25,0.00575,9.6000,3.2000,0.0033,0.0070,0.0040,0.0090,0.002852
1342,2.1110,0.1600,0.021000,0.0080,0.06100,0.0150,9.8600,0.17,0.1260,0.175000,...,1.466,0.25,0.01200,14.0610,5.1430,0.0040,0.1340,0.0040,0.1270,0.001000
1343,2.4840,0.1600,0.001000,0.0008,0.00000,0.0000,2.5180,0.17,0.2750,1.000000,...,0.220,0.25,0.02800,23.5000,10.0560,0.0010,0.0590,0.0061,0.2780,0.025000
1344,0.0900,0.1600,0.004000,0.0010,0.01600,0.0030,0.4240,0.17,0.0020,0.000995,...,10.775,0.25,0.00700,20.3960,5.6700,0.0080,0.0030,0.0090,0.0170,0.003000


In [ ]:
# Assuming df and pivot_df are your dataframes loaded earlier

# Loop for different values of k_SVD, K_knn, and StartCol
k_SVD_values = [10, 15, 20,30]  # Example values for k_SVD
K_knn_values = [3, 4,5, 6,7,8,9]  # Example values for K_knn
# change this to 3 without SCC hotcoding data
StartCol_values = [3]#[3, 4,5, 6]  # Example start columns

for StartCol in StartCol_values:
    for k_SVD in k_SVD_values:
        for K_knn in K_knn_values:
            X = pivot_df.iloc[:, StartCol:]

            # Apply SVD to X
            U, S, VT = svds(X.values, k_SVD)
            X_svd = U @ np.diag(S) @ VT
            X = X_svd

            # Compute KNN
            knn = NearestNeighbors(n_neighbors=K_knn)  # You can change the number of neighbors
            knn.fit(X)
            distances, indices = knn.kneighbors(X)

            # Create a graph
            G = nx.Graph()

            # Add nodes with PROFILE_NAME as labels
            for i, profile_name in enumerate(df['PROFILE_NAME']):
                G.add_node(i, label=profile_name)

            # Add edges based on KNN
            for i, neighbors in enumerate(indices):
                for neighbor in neighbors:
                    if i != neighbor:  # Avoid self-loops
                        G.add_edge(i, neighbor)

            # Apply Louvain algorithm
            coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)
            #print(coms)

            for i in range(len(coms.communities)):
                community = coms.communities[i]
                for j in community:
                  if j in com_map:
                    com_map[j].append(i)
                  else:
                    com_map[j]=[i]
            cluster_number = len(coms.communities)
            #print(cluster_number)
            print(f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} ")
            print_string =f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} "
            loop_info.append(print_string)


Current StartCol: 3, Current k_SVD: 10, Current K_knn: 3, No. of Clusters: 75 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 4, No. of Clusters: 44 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 5, No. of Clusters: 35 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 6, No. of Clusters: 31 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 7, No. of Clusters: 25 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 8, No. of Clusters: 23 
Current StartCol: 3, Current k_SVD: 10, Current K_knn: 9, No. of Clusters: 22 
Current StartCol: 3, Current k_SVD: 15, Current K_knn: 3, No. of Clusters: 73 
Current StartCol: 3, Current k_SVD: 15, Current K_knn: 4, No. of Clusters: 40 
Current StartCol: 3, Current k_SVD: 15, Current K_knn: 5, No. of Clusters: 32 
Current StartCol: 3, Current k_SVD: 15, Current K_knn: 6, No. of Clusters: 28 
Current StartCol: 3, Current k_SVD: 15, Current K_knn: 7, No. of Clusters: 24 
Current StartCol: 3, Current k_SVD: 15, Current K_kn

In [ ]:
import csv
# Writing com_map to CSV
with open('com_map_KNNImputer_wSCC.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Node', 'Communities'])
    for key, value in com_map.items():
        writer.writerow([key, ', '.join(map(str, value))])


In [ ]:
with open('loop_info_KNNImputer_wSCC.txt', mode='w') as file:
    for line in loop_info:
        file.write(line + '\n')

In [ ]:
com_map

{5: [0,
  11,
  10,
  1,
  0,
  1,
  1,
  11,
  3,
  6,
  7,
  1,
  8,
  5,
  10,
  12,
  9,
  4,
  9,
  2,
  3,
  10,
  12,
  14,
  6,
  7,
  10,
  2],
 90: [0,
  11,
  10,
  1,
  0,
  1,
  1,
  11,
  3,
  6,
  7,
  1,
  8,
  5,
  10,
  12,
  9,
  4,
  9,
  2,
  3,
  10,
  12,
  14,
  6,
  7,
  10,
  2],
 92: [0,
  4,
  10,
  5,
  0,
  10,
  1,
  22,
  8,
  12,
  15,
  1,
  11,
  12,
  53,
  11,
  8,
  7,
  2,
  2,
  3,
  53,
  9,
  7,
  10,
  3,
  11,
  2],
 93: [0,
  4,
  10,
  5,
  0,
  10,
  1,
  22,
  8,
  12,
  15,
  1,
  11,
  12,
  53,
  11,
  8,
  7,
  2,
  2,
  3,
  53,
  9,
  7,
  10,
  3,
  11,
  2],
 98: [0,
  7,
  13,
  1,
  13,
  1,
  14,
  15,
  15,
  4,
  6,
  12,
  0,
  11,
  25,
  17,
  13,
  5,
  16,
  9,
  6,
  23,
  10,
  15,
  8,
  16,
  8,
  14],
 106: [0,
  4,
  10,
  5,
  0,
  1,
  1,
  22,
  8,
  12,
  15,
  1,
  11,
  11,
  4,
  11,
  8,
  7,
  2,
  2,
  3,
  3,
  9,
  7,
  10,
  3,
  11,
  2],
 107: [0,
  4,
  10,
  5,
  0,
  10,
  1,
  22,
  8,
  12,
  15